<a href="https://colab.research.google.com/github/oluwafemidiakhoa/MLprject/blob/main/AdvancedAgile.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install openai pandas streamlit


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.7/360.7 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 52.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.0 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [13]:
from openai import OpenAI
import tkinter as tk
import re  # Import the re module for regular expressions
import traceback
from collections import defaultdict

# Initialize the OpenAI client
client = OpenAI(api_key="")


class Agent:
    def __init__(self, name):
        self.name = name

    def perform_task(self, task):
        raise NotImplementedError("Each agent must implement this method.")

class ProductManager(Agent):
    def create_product_backlog(self):
        backlog = ["Design UI", "Develop Feature A"]  # Removed "Test Feature A" from the backlog
        print(f"{self.name} created the product backlog: {backlog}")
        return backlog

class Developer(Agent):
    def __init__(self, name):
        super().__init__(name)

    def develop(self, task):
        if task == "Design UI":
            return self.develop_ui()
        elif task == "Develop Feature A":
            return self.develop_feature_a()
        else:
            raise ValueError(f"Unknown task: {task}")

    def develop_ui(self):
        prompt = "Generate Python code for a simple Tkinter UI with a label and a button."
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a software developer."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=150
        )
        code = response.choices[0].message.content.strip()
        print(f"{self.name} developed the following code for Design UI:\n{code}")
        return code

    def develop_feature_a(self):
        prompt = "Generate Python code for a function that adds 10 to the input data and returns the result."
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a software developer."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=150
        )
        raw_code = response.choices[0].message.content.strip()

        # Extract the Python code block using regex
        match = re.search(r"```python\n(.*?)\n```", raw_code, re.DOTALL)
        if match:
            code = match.group(1)
        else:
            # If no code block is found, handle as a failure
            print(f"Error: No valid Python code found in the response.\nResponse was:\n{raw_code}")
            code = ""

        print(f"{self.name} developed the following code for Develop Feature A:\n{code}")
        return code

class Tester(Agent):
    def test(self, task, code):
        if task == "Design UI":
            return self.test_ui(code)
        elif task == "Develop Feature A":
            return self.test_feature_a(code)
        else:
            raise ValueError(f"Unknown task: {task}")

    def test_ui(self, code):
        print(f"{self.name} is testing the UI code.")
        test_result = "Passed" if "tk.Tk()" in code and "label" in code and "button" in code else "Failed"
        print(f"Test result for Design UI: {test_result}")
        return test_result

    def test_feature_a(self, code):
        print(f"{self.name} is testing the code for Develop Feature A.")
        print(f"Code to be executed:\n{repr(code)}\n")  # Use repr to see any hidden characters
        try:
            # Replace \n with actual newlines to fix the syntax
            cleaned_code = code.replace("\\n", "\n")
            print(f"Cleaned code:\n{cleaned_code}\n")

            # Now attempt to execute the cleaned code
            exec_globals = {}
            exec(cleaned_code, exec_globals)
            # Example input to test the function
            test_input = 5
            expected_output = 15
            actual_output = exec_globals['add_ten'](test_input)
            test_result = "Passed" if actual_output == expected_output else f"Failed: expected {expected_output}, got {actual_output}"
        except SyntaxError as e:
            test_result = f"Failed due to syntax error: {str(e)}"
        except Exception as e:
            test_result = f"Failed due to error: {traceback.format_exc()}"
        print(f"Test result for Develop Feature A: {test_result}")
        return test_result

class AgileCoder:
    def __init__(self):
        self.agents = {
            "Product Manager": ProductManager("PM"),
            "Developer": Developer("Dev"),
            "Tester": Tester("Tester")
        }
        self.sprint_backlog = []
        self.codebase = defaultdict(str)

    def run_sprint(self):
        self.sprint_backlog = self.agents["Product Manager"].create_product_backlog()

        for task in self.sprint_backlog:
            code = self.agents["Developer"].develop(task)
            self.codebase[task] = code

            test_result = self.agents["Tester"].test(task, code)
            if "Failed" in test_result:
                print(f"Task {task} failed during testing. Sprint incomplete.")
                return

        print("Sprint complete!")
        print("Final codebase:", self.codebase)

# Initialize and run AGILECODER
agile_coder = AgileCoder()
agile_coder.run_sprint()







PM created the product backlog: ['Design UI', 'Develop Feature A']
Dev developed the following code for Design UI:
Sure! Here's a simple Python code for a Tkinter UI with a label and a button:

```python
import tkinter as tk

def button_click():
    label.config(text="Button clicked!")

root = tk.Tk()
root.title("Simple Tkinter UI")

label = tk.Label(root, text="Hello, world!")
label.pack()

button = tk.Button(root, text="Click me", command=button_click)
button.pack()

root.mainloop()
```

You can run this code in a Python environment with Tkinter installed to see the UI with a label displaying "Hello, world!" and a button that changes the label text when clicked.
Tester is testing the UI code.
Test result for Design UI: Passed
Dev developed the following code for Develop Feature A:
def add_ten(input_data):
    result = input_data + 10
    return result
Tester is testing the code for Develop Feature A.
Code to be executed:
'def add_ten(input_data):\n    result = input_data + 10\n    re

In [20]:
import re
from openai import OpenAI
import traceback
from collections import defaultdict, deque
import logging
import os

# Configure logging and use print statements for direct output
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')


# Initialize the OpenAI client
client = OpenAI(api_key="")

class Agent:
    def __init__(self, name):
        self.name = name

    def perform_task(self, task):
        raise NotImplementedError("Each agent must implement this method.")

class ProductManager(Agent):
    def create_product_backlog(self):
        backlog = deque(["Design UI", "Develop Feature A"])
        print(f"{self.name} created the product backlog: {list(backlog)}")
        return backlog

class Developer(Agent):
    def __init__(self, name):
        super().__init__(name)
        self.experience_level = 'Senior'

    def develop(self, task):
        print(f"{self.name} is working on the task: {task}")
        if task == "Design UI":
            return self.develop_ui()
        elif task == "Develop Feature A":
            return self.develop_feature_a()
        else:
            raise ValueError(f"Unknown task: {task}")

    def develop_ui(self):
        prompt = "Generate only valid Python code for a production-ready Tkinter UI with a label and a button, following best practices. Do not include any explanations or text outside the code."
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "You are a senior AI engineer specializing in UI development."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=200
        )
        full_response = response.choices[0].message.content.strip()
        print(f"Full response for Design UI:\n{full_response}")
        code = self.extract_code(full_response)
        print(f"{self.name} developed the following code for Design UI:\n{code}")
        return code

    def develop_feature_a(self):
        prompt = "Generate only valid Python code for a function that adds 10 to an input and handles edge cases. Do not include any explanations or text outside the code."
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "You are a senior AI engineer."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=200
        )
        full_response = response.choices[0].message.content.strip()
        print(f"Full response for Develop Feature A:\n{full_response}")
        code = self.extract_code(full_response)
        print(f"{self.name} developed the following code for Develop Feature A:\n{code}")
        return code

    def extract_code(self, raw_response):
        # Attempt to extract Python code blocks
        matches = re.findall(r"```python\n(.*?)\n```", raw_response, re.DOTALL)
        if matches:
            # Join all extracted code blocks (if multiple)
            code = "\n".join(matches)
            print("Extracted Python code block:\n", code)
        else:
            print("WARNING: No valid Python code found in the response. Returning raw response.")
            code = raw_response
        return code

class Tester(Agent):
    def __init__(self, name):
        super().__init__(name)
        self.experience_level = 'Expert'

    def test(self, task, code):
        print(f"{self.name} is testing the code for task: {task}")
        if task == "Design UI":
            return self.test_ui(code)
        elif task == "Develop Feature A":
            return self.test_feature_a(code)
        else:
            raise ValueError(f"Unknown task: {task}")

    def test_ui(self, code):
        try:
            exec_globals = {}
            print(f"Executing the following code for UI:\n{code}")
            exec(code, exec_globals)
            if 'tk.Tk' in code and 'label' in code and 'button' in code:
                test_result = "Passed"
            else:
                test_result = "Failed"
            print(f"Test result for Design UI: {test_result}")
        except Exception as e:
            print(f"ERROR: Test for Design UI failed due to an error: {traceback.format_exc()}")
            test_result = "Failed"
        return test_result

    def test_feature_a(self, code):
        try:
            cleaned_code = code.replace("\\n", "\n")
            exec_globals = {}
            print(f"Executing the following code for Feature A:\n{cleaned_code}")
            exec(cleaned_code, exec_globals)
            test_input = 5
            expected_output = 15
            actual_output = exec_globals['add_ten'](test_input)
            test_result = "Passed" if actual_output == expected_output else f"Failed: expected {expected_output}, got {actual_output}"
            print(f"Test result for Develop Feature A: {test_result}")
        except Exception as e:
            print(f"ERROR: Test for Develop Feature A failed due to an error: {traceback.format_exc()}")
            test_result = "Failed"
        return test_result

class AgileCoder:
    def __init__(self):
        self.agents = {
            "Product Manager": ProductManager("PM"),
            "Developer": Developer("Dev"),
            "Tester": Tester("Tester")
        }
        self.sprint_backlog = deque()
        self.codebase = defaultdict(str)

    def run_sprint(self):
        self.sprint_backlog = self.agents["Product Manager"].create_product_backlog()

        while self.sprint_backlog:
            task = self.sprint_backlog.popleft()
            try:
                code = self.agents["Developer"].develop(task)
                self.codebase[task] = code

                test_result = self.agents["Tester"].test(task, code)
                if "Failed" in test_result:
                    print(f"ERROR: Task {task} failed during testing. Sprint incomplete.")
                    return

            except Exception as e:
                print(f"ERROR: Task {task} failed due to an error: {traceback.format_exc()}")
                return

        print("Sprint complete!")
        print(f"Final codebase:\n{dict(self.codebase)}")

# Initialize and run AGILECODER
agile_coder = AgileCoder()
agile_coder.run_sprint()


PM created the product backlog: ['Design UI', 'Develop Feature A']
Dev is working on the task: Design UI
Full response for Design UI:
```python
import tkinter as tk

class Application(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Tkinter UI")
        self.geometry("300x200")

        self.create_widgets()

    def create_widgets(self):
        self.label = tk.Label(self, text="This is a label!")
        self.label.pack()

        self.button = tk.Button(self, text="This is a button!", command=self.button_click)
        self.button.pack()

    def button_click(self):
        print("Button clicked!")

def main():
    app = Application()
    app.mainloop()

if __name__ == "__main__":
    main()
```
Please replace `print("Button clicked!")` inside the `button_click` function with your desired business logic.
Extracted Python code block:
 import tkinter as tk

class Application(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Tk